# Using finetuned model

Created the fine tuned model for version 1 in prep_dataset.ipynb

This is to check if it works, and see how well it works

In [6]:
from transformers import AutoTokenizer, GenerationConfig, LlamaForCausalLM, LlamaTokenizer
import torch
from peft import PeftModel

base_model = "meta-llama/Llama-2-7b-hf"

load_8bit: bool = False
base_model: str = ""
lora_weights: str = "tloen/alpaca-lora-7b"
prompt_template: str = ""  # The prompt template to use, will default to alpaca.
server_name: str = "0.0.0.0"  # Allows to listen on all interfaces by providing '0.
share_gradio: bool = False

In [9]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: ''.

In [ ]:

model = LlamaForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=load_8bit,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(
    model,
    "./results",
    torch_dtype=torch.float16,
)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    "./results",
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True
)

In [1]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

model_name = "decapoda-research/llama-7b-hf"
adapters_name = "lucas0/empath-llama-7b"

model_name = "meta-llama/Llama-2-7b-hf"
adapters_name = "/home/msaad/workspace/honors-thesis/fine-tuning/results/final_checkpoint"

print(f"Starting to load the model {model_name} into memory")

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)
m = PeftModel.from_pretrained(m, adapters_name)
m = m.merge_and_unload()
tok = LlamaTokenizer.from_pretrained(model_name)
tok.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/msaad/miniconda3/envs/thesis/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /home/msaad/miniconda3/envs/thesis/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/msaad/miniconda3/envs/thesis/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/msaad/miniconda3/envs/thesis/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/msaad/miniconda3/envs/thesis/lib/libcudart.so'), PosixPath('/home/msaad/miniconda3/envs/thesis/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


Starting to load the model meta-llama/Llama-2-7b-hf into memory


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Successfully loaded the model meta-llama/Llama-2-7b-hf into memory


In [5]:
text = "How do I apply?"
inputs = tok(text, return_tensors="pt").to("cuda")
outputs = m.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=50, pad_token_id=tok.eos_token_id)

print(tok.decode(outputs[0], skip_special_tokens=True))

How do I apply?
